In [1]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_validate
import pickle

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [2]:
data = arff.load(open('../text-blob-pt/OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.33, random_state=42)

pt_stop_words = nltk.corpus.stopwords.words('portuguese')

classifiers = []
max_df = 0.6

In [3]:
class CustomClassifier(BaseEstimator, ClassifierMixin):
    """An example of classifier"""
    
    def __init__(self):
        self.mlp = Pipeline([
            ('tfidf', TfidfVectorizer(max_df=0.2,
                                      min_df=5,
                                      ngram_range=(1, 2),
                                      smooth_idf=True,
                                      strip_accents='ascii',
                                      sublinear_tf=False,
                                      use_idf=False)),
            ('clf', MLPClassifier(activation='logistic', alpha=0, solver='lbfgs')),
        ])
        self.rf = Pipeline([
            ('tfidf', TfidfVectorizer(max_df=0.2,
                                      min_df=2,
                                      ngram_range=(1, 1),
                                      smooth_idf=False,
                                      strip_accents='ascii',
                                      sublinear_tf=True,
                                      use_idf=True)),
            ('clf', RandomForestClassifier(max_depth=None,
                                           min_samples_leaf=1, min_samples_split=2, n_estimators=100)),
        ])

        self.mnb =  Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.6,
                                 min_df= 5,
                                 ngram_range= (1, 1),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', MultinomialNB(alpha=0.1, fit_prior=False)),
        ])

        self.svc = Pipeline([
            ('tfidf', TfidfVectorizer(max_df=0.6,
                                      min_df=1,
                                      ngram_range=(1, 1),
                                      smooth_idf=True,
                                      strip_accents='ascii',
                                      sublinear_tf=False,
                                      use_idf=True)),
            ('clf', SVC(C=4, kernel='linear', probability=True, shrinking=True, tol=1)),
        ])
        
        self.dt = Pipeline([
        ('tfidf',TfidfVectorizer(max_df = 0.6,
                                 min_df = 1,
                                 ngram_range = (1, 3),
                                 smooth_idf = True,
                                 strip_accents = 'ascii',
                                 sublinear_tf = False,
                                 use_idf = True)),
        ('clf', DecisionTreeClassifier(class_weight= {0: 1, 1: 2}, criterion= 'gini', min_samples_split = 3)),
        ])
        
        self.sgd = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 10,
                                 ngram_range= (1, 3),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = False)),
        ('clf', SGDClassifier(alpha=0.01, loss='perceptron', penalty='none', tol=0.001, max_iter=1000)),
        ])

        self.classifiers = [self.rf, self.mnb, self.svc, self.mlp, self.dt, self.sgd]

    def fit(self, X, y=None):
        for classifier in self.classifiers:
            classifier.fit(X, y)
        return self

    def predict(self, X, y=None):
        try:
            results = []
            for classifier in self.classifiers:
                results.append(classifier.predict(X))
            
            result = []
            for i in range(len(X)): # Mudar aqui
                partial_result = (results[0][i] or results[1][i] or results[2][i] 
                                or results[3][i] or results[4][i] or results[5][i])
                result.append(partial_result)

        except AttributeError:
            raise RuntimeError(
                "You must train classifer before predicting data!")

        return result
    

    def score(self, X, y=None):
        return(sum(self.predict(X)))


In [11]:
custom = CustomClassifier()
custom.fit(X_train, y_train)
start_time = time.time()
pred = custom.predict(X_test)
print("time: ", (time.time() - start_time))
report = classification_report(y_test, pred, output_dict=True)
# Rodar até aqui
, self.mlp, self.dt, self.sgd

time:  0.1821422576904297


('self.mlp,', 'self.dt,', 'self.sgd')

### mnb - rf - svc -> Voto minoria

In [ ]:
# copiar esse código e colar numa célular vazia
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

### mnb - mlp - svc -> Voto minoria

In [11]:
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

              precision    recall  f1-score   support

           0       0.85      0.79      0.82       267
           1       0.40      0.51      0.45        74

   micro avg       0.73      0.73      0.73       341
   macro avg       0.63      0.65      0.63       341
weighted avg       0.76      0.73      0.74       341

Accuracy: 0.64 (+/- 0.07) [voting]
Recall: 0.64 (+/- 0.13) [voting]
Precision: 0.31 (+/- 0.07) [voting]


### mnb - mlp - svc -> soft

In [41]:
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       267
           1       0.42      0.35      0.38        74

   micro avg       0.75      0.75      0.75       341
   macro avg       0.62      0.61      0.61       341
weighted avg       0.74      0.75      0.75       341

Accuracy: 0.72 (+/- 0.05) [voting]
Recall: 0.41 (+/- 0.11) [voting]
Precision: 0.37 (+/- 0.13) [voting]


### rf or (mlp and svc)

In [44]:
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       267
           1       0.52      0.45      0.48        74

   micro avg       0.79      0.79      0.79       341
   macro avg       0.69      0.67      0.68       341
weighted avg       0.78      0.79      0.79       341

Accuracy: 0.77 (+/- 0.04) [voting]
Recall: 0.45 (+/- 0.11) [voting]
Precision: 0.48 (+/- 0.15) [voting]


In [47]:
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       267
           1       0.46      0.47      0.47        74

   micro avg       0.77      0.77      0.77       341
   macro avg       0.66      0.66      0.66       341
weighted avg       0.77      0.77      0.77       341

Accuracy: 0.72 (+/- 0.05) [voting]
Recall: 0.52 (+/- 0.11) [voting]
Precision: 0.38 (+/- 0.08) [voting]


In [25]:
print(classification_report(y_test, pred))
scores = cross_val_score(custom, X, y, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='recall')
print("Recall: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))
scores = cross_val_score(custom, X, y, cv=10, scoring='precision')
print("Precision: %0.2f (+/- %0.2f) [%s]" 
        % (scores.mean(), scores.std(), 'voting'))

              precision    recall  f1-score   support

           0       0.88      0.51      0.65       267
           1       0.30      0.74      0.42        74

   micro avg       0.56      0.56      0.56       341
   macro avg       0.59      0.63      0.54       341
weighted avg       0.75      0.56      0.60       341

Accuracy: 0.53 (+/- 0.10) [voting]
Recall: 0.77 (+/- 0.11) [voting]
Precision: 0.27 (+/- 0.05) [voting]
